In [60]:
import numpy as np
%load_ext Cython

The Cython extension is already loaded. To reload it, use:
  %reload_ext Cython


In [61]:
"""
Most of the BJ stuff should follow general game conventions. 
We initially model more intricate details (burning, shuffling protocol) based on local casino.
https://www.crownmelbourne.com.au/getsydmedia/99be12f2-c48b-44dc-b6ba-ab5849f6f723/crown-melbourne-gaming-blackjack-rules_1.pdf?ext=.pdf
"""

# configurations
NDECKS: int = 8
# NPLAYERS: int = 4 # affects how many cards you may be able to count
DECK_PENETRATION: int = 2 # typical number of decks dealt out before reshuffle

In [68]:
"""
We keep our representation of the deck and cards as characters, rather than grouping on BJ value
Importantly, this means our framework remains flexible enough to add in weird house rules such as perfect pairs/flushes
"""
cards = np.array(['2','3','4','5','6','7','8','9','T','J','Q','K','A']) # use char T for 10 
suits = np.array(['D', 'S', 'C', 'H'])
deck = np.array([np.char.add(cards, suit) for suit in suits]).ravel()
shoe = np.repeat(deck, NDECKS)

# We use bytestrings as our dtype so we can have a Cython memview over the numpy card shoe of C char type
shoe = shoe.astype('S2')

numcardsdeal = len(deck) * DECK_PENETRATION

In [63]:
rng = np.random.default_rng() # https://numpy.org/doc/stable/reference/random/index.html
%timeit rng.shuffle(shoe)

9.73 µs ± 104 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


In [70]:
# bad idea to use value mapping after every shuffle
values = np.array([2,3,4,5,6,7,8,9,10,10,10,10,11])
dict_map = dict(zip(cards.astype('S1'), values))
%timeit np.array([dict_map[x[:1]] for x in shoe]) # slicing byte string to get first char
# %timeit np.vectorize(dict_map.get)(shoe)

93.1 µs ± 3.26 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [71]:
import pandas as pd
strat = pd.read_csv(r"StrategyInput\BasicNoDeviations-4to8Decks-HitSoft17.csv", header=None)

"""
Read in the individual strategy tables, and set top left corner position to 0.
"""
hard = strat.iloc[0:18, :].copy().reset_index(drop=True)
hard[0][0] = '0'
hard = hard.values.astype(str)

soft = strat.iloc[18:27, :].copy().reset_index(drop=True)
soft[0][0] = '0'
soft = soft.values.astype(str)

splits = strat.iloc[27:38, :].copy().reset_index(drop=True)
splits[0][0] = '0'
splits = splits.values.astype(str)

In [ ]:
soft

array([['0', '2', '3', '4', '5', '6', '7', '8', '9', 'T', 'A'],
       ['13', 'H', 'H', 'H', 'D', 'D', 'H', 'H', 'H', 'H', 'H'],
       ['14', 'H', 'H', 'H', 'D', 'D', 'H', 'H', 'H', 'H', 'H'],
       ['15', 'H', 'H', 'D', 'D', 'D', 'H', 'H', 'H', 'H', 'H'],
       ['16', 'H', 'H', 'D', 'D', 'D', 'H', 'H', 'H', 'H', 'H'],
       ['17', 'H', 'D', 'D', 'D', 'D', 'H', 'H', 'H', 'H', 'H'],
       ['18', 'D', 'D', 'D', 'D', 'D', 'S', 'S', 'H', 'H', 'H'],
       ['19', 'S', 'S', 'S', 'S', 'D', 'S', 'S', 'S', 'S', 'S'],
       ['20', 'S', 'S', 'S', 'S', 'S', 'S', 'S', 'S', 'S', 'S']],
      dtype='<U2')

In [76]:
%%cython 
import numpy as np
cimport cython 

# Create a packed struct in C representing one card in the shoe
# https://cython.readthedocs.io/en/latest/src/userguide/memoryviews.html
cdef packed struct card_dtype_struct: 
    char face
    char suit

# C uses ascii integer representation for characters
# Numeric characters in our shoe will have an ascii value less than ascii of 'a'
# English characters will have an ascii value greater than ascii of 'a'
# https://www.cs.cmu.edu/~pattis/15-1XX/common/handouts/ascii.html
cdef int ASCII_A = 65 

@cython.boundscheck(False)
@cython.wraparound(False)
@cython.initializedcheck(False)
def play_shoe(card_dtype_struct[:] shoe not None, int numcardsdeal, char[:,:] hrd not None, 
                char[:,:] sft not None, char[:,:] splt not None):
    """
    Simulate playing one shuffled shoe, until a cut card is drawn. 
    For now, return to main Python loop for reshuffle. 
    """
    cdef Py_ssize_t dealt = 0 # track what has been 'dealt' from shoe
    cdef int count = 0 # card count
    
    # this should roughly simulate a 'cut card'
    # we have to exceed the cut position prior to dealing
    while dealt < numcardsdeal:
        




In [79]:
play_shoe(shoe, numcardsdeal, hard[1:,1:].astype('S1'), soft[1:,1:].astype('S1'), splits[1:,1:].astype('S1'))

{'face': 50, 'suit': 68}
72


In [ ]:
shoe[:50]

array([b'QS', b'9S', b'8S', b'TS', b'9C', b'JS', b'4S', b'5H', b'6S',
       b'6S', b'8C', b'JH', b'5H', b'2H', b'7H', b'JC', b'QS', b'6H',
       b'KH', b'7C', b'JS', b'5D', b'KD', b'2H', b'9C', b'4H', b'AS',
       b'KC', b'6C', b'6S', b'QC', b'8D', b'4D', b'5C', b'7H', b'9D',
       b'QH', b'2H', b'7S', b'TD', b'TS', b'5C', b'8D', b'KS', b'7C',
       b'AC', b'9D', b'TH', b'JS', b'8D'], dtype='|S2')

In [78]:
hard[1:,1:]

array([['H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H'],
       ['H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H'],
       ['H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H'],
       ['H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H'],
       ['H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H'],
       ['H', 'D', 'D', 'D', 'D', 'H', 'H', 'H', 'H', 'H'],
       ['D', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'H', 'H'],
       ['D', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'D'],
       ['H', 'H', 'S', 'S', 'S', 'H', 'H', 'H', 'H', 'H'],
       ['S', 'S', 'S', 'S', 'S', 'H', 'H', 'H', 'H', 'H'],
       ['S', 'S', 'S', 'S', 'S', 'H', 'H', 'H', 'H', 'H'],
       ['S', 'S', 'S', 'S', 'S', 'H', 'H', 'H', 'R', 'R'],
       ['S', 'S', 'S', 'S', 'S', 'H', 'H', 'R', 'R', 'R'],
       ['S', 'S', 'S', 'S', 'S', 'S', 'S', 'S', 'S', 'R'],
       ['S', 'S', 'S', 'S', 'S', 'S', 'S', 'S', 'S', 'S'],
       ['S', 'S', 'S', 'S', 'S', 'S', 'S', 'S', 'S', 'S'],
       ['S', 'S', 'S', 'S', 'S', 'S', 'S', 'S', 'S', 'S'